In [2]:
trainlines = open("datasets/msr-med/train.tsv","r").readlines()
vallines = open("datasets/msr-med/val.tsv","r").readlines()
testlines = open("datasets/msr-med/test.tsv","r").readlines()
trainlines.extend(vallines)
trainlines.extend(testlines)

In [3]:
def create_binary_dataset(alllines):
    trainfile = open("datasets/ranking-med/train.tsv","w+")
    valfile = open("datasets/ranking-med/val.tsv","w+")
    testfile = open("datasets/ranking-med/test.tsv","w+")
    
    bertfile = open("datasets/bert_file_new.txt","w+")
    
    sentence_lengths = []
    query_lengths = []
    
    bert_lines = []
    
    all_rlines = []
    for line in alllines:
        line = line.strip().split("\t")
        idx = line[0]
        query = line[1]
        choices= line[2:-1]
        label = line[-1]
        correct_choice = choices[int(label)]
        
        bertfile.write(query+"\n")
        
        sentence_lengths.append(len(correct_choice.split(" ")))
        query_lengths.append(len(query.split(" ")))
        
        wrong_choices = []
        for choice in choices:
            if choice != correct_choice:
                wrong_choices.append(choice)
            bertfile.write(choice+"\n")
        
        bertfile.write("\n")
        
        for cidx,choice in enumerate(wrong_choices[0:5]):
            nidx=idx+"-"+str(cidx)
            nlabel = "0"
            all_rlines.append(nidx+"\t"+query+"\t"+choice+"\t"+nlabel+"\n")
        for cidx in range(5,10):
            nidx=idx+"-"+str(cidx)
            nlabel = "1"
            all_rlines.append(nidx+"\t"+query+"\t"+correct_choice+"\t"+nlabel+"\n")
            
    total_len  = len(all_rlines)
    train_size = int(total_len*0.7)
    val_size = int(total_len*0.15)
    print("Total Size:",total_len,"Train:Size",train_size,"Val Size",val_size)
    
    trainlines = all_rlines[0:train_size]
    vallines = all_rlines[train_size:train_size+val_size]
    testlines = all_rlines[train_size+val_size:]
    
    for line in trainlines:
        trainfile.write(line)
    trainfile.close()
    
    for line in vallines:
        valfile.write(line) 
    valfile.close()
    
    for line in testlines:
        testfile.write(line) 
    testfile.close()
    
    bertfile.close()
    
    return sentence_lengths,query_lengths

In [4]:
slens,qlens = create_binary_dataset(trainlines)

Total Size: 3341880 Train:Size 2339316 Val Size 501282


In [5]:
import numpy as np

In [6]:
np.percentile(slens,95),np.percentile(qlens,95)

(105.0, 10.0)

In [7]:
a=np.zeros([768])
b=np.ones([768])

In [8]:
from scipy import stats

In [9]:
stats.percentileofscore(qlens,28)

99.98907800399775

In [10]:
import torch

In [11]:
a = torch.Tensor(a)
b = torch.Tensor(b)

In [12]:
b[2]

tensor(1.)